In [ ]:
from collections import deque

def bfs(graph, start):
    visited = [False] * len(graph)
    queue = deque([start])

    while queue:
        vertex = queue.popleft()
        if not visited[vertex]:
            print(vertex, end=" ")
            visited[vertex] = True
            for neighbor in graph[vertex]:
                if not visited[neighbor]:
                    queue.append(neighbor)


graph = {0: [1, 3], 1: [0, 2], 2: [1, 4], 3: [0, 4], 4: [2, 3,5],5:[4]}

start_vertex = 0

bfs(graph, start_vertex)

0 1 3 2 4 5 

In [ ]:
def dfs(graph, start):
    visited = set()
    stack = [start]

    while stack:
        vertex = stack.pop()

        if vertex not in visited:
            print(vertex)
            visited.add(vertex)
            stack.extend([neighbor for neighbor in graph[vertex] if neighbor not in visited])


graph = {
    'A': ['B', 'S'],
    'B': ['A'],
    'C': ['D', 'E', 'F', 'S'],
    'D': ['C'],
    'E': ['C', 'H'],
    'F': ['C', 'G'],
    'G': ['F', 'S'],
    'H': ['E'],
    'S': ['A', 'C', 'G']
}
start_node = 'A'
dfs(graph, start_node)

A
S
G
F
C
E
H
D
B


In [6]:
from copy import deepcopy
import numpy as np
import time

def bestsolution(state):
    bestsol = np.array([], int).reshape(-1, 9)
    count = len(state) - 1
    while count != -1:
        bestsol = np.insert(bestsol, 0, state[count]['puzzle'], 0)
        count = state[count]['parent']
    return bestsol.reshape(-1, 3, 3)

def all_unique(checkarray):

    unique_elements = set()
    for item in checkarray:
        if item in unique_elements:
            return False
        unique_elements.add(item)
    return True

def misplaced_tiles(puzzle, goal):
    mscost = np.sum(puzzle != goal) - 1
    return mscost if mscost > 0 else 0

def coordinates(puzzle):
    pos = np.array(range(9))
    for p, q in enumerate(puzzle):
        pos[q] = p
    return pos

def evaluate_misplaced(puzzle, goal):
    steps = np.array([('up', [0, 1, 2], -3), ('down', [6, 7, 8], 3), ('left', [0, 3, 6], -1), ('right', [2, 5, 8], 1)],
                     dtype=[('move', str, 1), ('position', list), ('head', int)])
    dtstate = [('puzzle', list), ('parent', int), ('gn', int), ('hn', int)]
    costg = coordinates(goal)
    parent = -1
    gn = 0
    hn = misplaced_tiles(coordinates(puzzle), costg)
    state = np.array([(puzzle, parent, gn, hn)], dtstate)
    dtpriority = [('position', int), ('fn', int)]
    priority = np.array([(0, hn)], dtpriority)

    while 1:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])
        position, fn = priority[0]
        priority = np.delete(priority, 0, 0)
        puzzle, parent, gn, hn = state[position]
        puzzle = np.array(puzzle)
        blank = int(np.where(puzzle == 0)[0])
        gn = gn + 1
        c = 1
        start_time = time.time()

        for s in steps:
            c = c + 1
            if blank not in s['position']:
                openstates = deepcopy(puzzle)
                openstates[blank], openstates[blank + s['head']] = openstates[blank + s['head']], openstates[blank]
                if ~(np.all(list(state['puzzle']) == openstates, 1)).any():
                    end_time = time.time()
                    if (end_time - start_time) > 2:
                        print("The 8 puzzle is unsolvable\n")
                        break
                    hn = misplaced_tiles(coordinates(openstates), costg)
                    q = np.array([(openstates, position, gn, hn)], dtstate)
                    state = np.append(state, q, 0)
                    fn = gn + hn
                    q = np.array([(len(state) - 1, fn)], dtpriority)
                    priority = np.append(priority, q, 0)
                    if np.array_equal(openstates, goal):
                        print('The 8 puzzle is solvable\n')
                        return state, len(priority)
        if np.array_equal(openstates, goal):
            print('The 8 puzzle is solvable\n')
            return state, len(priority)
    return state, len(priority)


puzzle = [2, 8, 3, 7, 1, 4, 0, 6, 5]
goal = [1, 2, 3, 8, 0, 4, 7, 6, 5]
state, visited = evaluate_misplaced(puzzle, goal)
bestpath = bestsolution(state)
print(str(bestpath).replace('[', '').replace(']', '').replace('  ', ''))
totalmoves = len(bestpath) - 1
print('\nSteps to reach goal:', totalmoves)
visit = len(state) - visited
print('Total nodes visited: ', visit, "\n")

The 8 puzzle is solvable

2 8 3
7 1 4
0 6 5

 2 8 3
0 1 4
7 6 5

 2 8 3
1 0 4
7 6 5

 2 0 3
1 8 4
7 6 5

 0 2 3
1 8 4
7 6 5

 1 2 3
0 8 4
7 6 5

 1 2 3
8 0 4
7 6 5

Steps to reach goal: 6
Total nodes visited:  11 

